In [8]:
from langchain.llms import OpenAI
llm = OpenAI(temperature = 0.5) # temperature is a measure of randomness
#print(llm.predict('How old was donald trump in 1987?'))


In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (AIMessage,HumanMessage,SystemMessage)

chat = ChatOpenAI(temperature = 0.5)
#print(chat.predict_messages([HumanMessage(content="whats 9+10 (hint, the answer I want is referencing a meme)")]))

In [5]:
chat.predict("whats 9+10 (hint, the answer I want is referencing a meme)")

'21 (the answer from the "What\'s 9+10?" meme)'

In [7]:
from langchain.prompts import PromptTemplate

genericTemplate = PromptTemplate.from_template("What is the best {amenity} in {location} for {reason}")
gymPrompt = template.format(amenity = "gym", location = "Soho", reason = "athletes")
chat.predict(gymPrompt)


"As an AI language model, I do not have personal preferences or opinions. However, based on online reviews and ratings, some of the best gyms in Soho for athletes are:\n\n1. Equinox Soho: This gym offers a range of equipment and classes for athletes, including strength training, cardio, and yoga. It also has a spacious and well-maintained facility.\n\n2. Barry's Bootcamp: This gym is known for its high-intensity interval training (HIIT) classes that are designed to challenge athletes and improve their fitness levels.\n\n3. SoulCycle Soho: This gym offers indoor cycling classes that are popular among athletes looking to improve their endurance and cardiovascular health.\n\n4. Fhitting Room: This gym offers group classes that combine strength training and HIIT workouts, making it a great option for athletes looking for a challenging and varied workout.\n\n5. Rumble Boxing: This gym offers boxing-inspired workouts that are great for athletes looking to improve their strength, agility, and

**Playing with Chains**\
These pass your input first to a PromptTemplate which is then passed to an LLM. 

In [14]:
from langchain.chains import LLMChain

chain = LLMChain(llm = llm, prompt = prompt)
#chain.run({'amenity':"gym", 'location':"Soho", 'reason':"athletes"})
chain.run(amenity= "gym", location= "Soho", reason= "athletes")

'?\n\nThe best gym in Soho for athletes is Equinox Soho. It offers a wide range of equipment and classes for athletes, as well as a wide variety of amenities. It also has a team of expert trainers to help you reach your fitness goals.'

**Roles**
- Notice how in the following example, we use more specific roles (chat, system & human), rather than just the generic template we used previously (these are specifc to openAI)
- The 'System' role can be used to declare messages that determine the behaviour/ personality of the 'Assistant'. For example, your typical "You are XGPT" type of prompts. 
- The 'User' sends requests for the 'Assistant' to respond to.\

**Why Chat Models?**\
The APIs are stateless, so the inputs you send are all the context the LLM will have. Chats act as a memory of sorts, collating all neccessary context you'd want based on a series of messages/prompts, e.g.:

**And What about Chains?**\
Chains provide a way to link the various primitives surrounding LLMs (e.g. prompts, models and other chains)
~~~
openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
~~~

In [20]:
from langchain.prompts.chat import (ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate)
GPT = ChatOpenAI(temperature=1)

system_template = "You are a linguistic guru that translates {input} to {output}"
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chain = LLMChain(llm=GPT, prompt = chat_prompt)
chain.run(input = "Spanish",output = "English", text = "Que pasa amigo?")

"What's up, friend?"